In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn

# Project: The energy transition in Switzerland

### Import Data

In [ ]:
# here we import the data
swiss_energy_path = 'Data/swiss_energy_production.xlsx'
metadata = pd.read_excel(swiss_energy_path, index_col = None, header = 2)

In [ ]:
#metadata = metadata.fillna(value=0)
metadata.head(10)

### Clean the Data

First we will translate the data

In [ ]:
# we translate the cols in english and add unit, either % or TJ (Tera Joule)
def translate(df):
    new_name = ["Wood and charcoal in TJ", "Wood and charcoal in %", "Hydraulic force in TJ", "Hydraulic force in %", "Industrial waste and scrap in TJ", "Industrial waste and scrap in %", "Coal in TJ","Coal in %","Crude oil and petroleum products in TJ","Crude oil and petroleum products in %","Gas in TJ","Gas in %","Nuclear fuels in TJ", "Nuclear fuels in %","Other renewable energies in TJ", "Other renewable energies in %", "Total use of energy agents in TJ", "Total use of energy agents in %","Electricity, balance import / export in TJ", "Electricity, balance import / export in %", "Gross energy consumption in the country = 100% in TJ", "Gross energy consumption in the country = 100% in %"]
    new_df = df.copy()
    cols_df = new_df.columns.values 
    for i in range(len(cols_df)):
        name = cols_df[i]
        new_df = new_df.rename(columns={name: new_name[i]})
    return new_df

In [ ]:
data = translate(metadata)
data.head(3)

From here we can already see that we have 2 kinds of data (percentage data and data representing consumption in tera joules). We will first clean all the data and then have different dataframes to better analyze the results. Furthermore, the plots will be more understandable and we will better see the evolution of every kind of energy during the years.

Then we will clean the data

In [ ]:
# check if a string is a number or not
def is_number(s):
    try:
        s = int(s)
        if(np.isnan(s)):
            return False
        else:
            return True
    except ValueError:
        return False

# We have some None values in index so we keep only the indexes that are dates
def keep_only_dates(df):
    new_df = df.copy()
    index_df = new_df.index.values
    bool_dates = []
    for i in range(len(index_df)):
        date = index_df[i]
        bool_dates.append(is_number(date))
        
    return new_df[bool_dates]

In [ ]:
data = keep_only_dates(data)
data.head(3)

Now we need to change de values where we have ... instead of 0 

In [ ]:
# Check if all index are ok
data.index.is_unique

In [ ]:
data_clean = data.apply(pd.to_numeric, errors = 'coerce').fillna(value=0)
data_clean.head(5)

We can see that we have a column "Gross energy consumption in the country = 100% in %" which is always equal to 100% and it's the same thing for "Total use of energy agents in %"... so it's not very interesting for us. <br> 
Futhermore the "Gross energy consumption in the country = 100% in TJ" is the same thing as "Total use of energy agents in TJ". So we will delete one and rename the other because it's a little bit ambiguous.

In [ ]:
data_clean_drop = data_clean.drop(['Gross energy consumption in the country = 100% in %'], axis=1).drop(['Total use of energy agents in %'], axis=1).drop(['Total use of energy agents in TJ'], axis=1)
final_data_clean = data_clean_drop.rename(columns={"Gross energy consumption in the country = 100% in TJ": "Gross energy consumption in the country in TJ"})
final_data_clean.head()

In [ ]:
# Check one
final_data_clean["Industrial waste and scrap in %"].head(5)

Now that we have this, we can separate our datafram into 2 differents. Why? So we can have one with only with Tera-joules and one with only the percentage

In [ ]:
def countains_str(df, str_):
    new_df = df.copy()
    index_df = new_df.columns.values
    names = []
    for i in range(len(index_df)):
        col_name = index_df[i]
        if str_ in col_name: 
            names.append(col_name)
    return new_df[names]

In [ ]:
final_data_tj_temp = countains_str(final_data_clean, "TJ")
final_data_tj_temp.head()

In [ ]:
final_data_perc = countains_str(final_data_clean, "%")
final_data_perc.head()

Finally we will store in 2 different data frame the "Consommation brute d’énergie dans le pays in TJ" and the rest of the dataframe. The reason for this is that the outliers from "Consommation brute d’énergie dans le pays in TJ" won't perturbe the others

In [ ]:
final_data_tj_full = final_data_tj_temp[["Gross energy consumption in the country in TJ"]]
final_data_tj_full.head()

In [ ]:
final_data_tj = final_data_tj_temp.drop(['Gross energy consumption in the country in TJ'], axis=1)
final_data_tj.head()

In [ ]:
final_data_tj.index.values

We can see that our data goes to 2014 and not 2016 so if we need to compare the data, we have to be careful

Now we have our clean data and we can analyse them

## Energy Production

First we will see by year the evolution of the enery production in Switzerland in Tera Joules

In [ ]:
plt.figure(); 
final_data_tj.plot();

In [ ]:
plt.figure(); 
final_data_tj['Gas in TJ'].plot();

However it's a little bit messy and we don't fully understand the plot. So for this we will use more plots

In [ ]:
def shared_plot(df):
    cols = df.columns.values
    half_nb_cols = int(len(cols)/2)
    cols1 = cols[:half_nb_cols]
    cols2 = cols[half_nb_cols:]
        
    fig, axs = plt.subplots(1, 2, figsize=(20,6), sharey=True)
    df[cols1].plot(ax=axs[0], title = "First half")
    df[cols2].plot(ax=axs[1], title = "Second half")
    
    plt.show()

In [ ]:
shared_plot(final_data_tj)

Here we can see the results better. Furthermore, the y axis is shared so the comparaison will be easier to do. For example we can that since 1945 the Swiss oil consumption has risen sharply as nuclear combustible since 1970.  Other energy like the coal and charcoal are almost gone. However for our project we are interested about the energy transition towards renewable energies. Here we can see that since approximately 2011 there is a transition. Indeed since Fukushima in 2011, we can see that Switzerland has begun its energetic transition. In addition, during a vote held in May 2017, the Swiss voted 58.2% in favor of a gradual withdrawal of nuclear power to the benefit of renewable energy, more respectful of the environment. <br>
Here we see the production in TJ for each energy but what about the Total energy consumed?

In [ ]:
final_data_tj_full.plot(title = "Total Enegry in TJ")

This confirms what was said earlier. Indeed we can see that after the war of 1945, the consumption of energy drastically increased because it is necessary to rebuild and to have the same status of the United States, the world's leading country. At that time, energy is the engine of development and that's why energy consumption increased a lot. The baby boom around 1965 can also explain this crazy energy consumption. However we see more and more that the energy consumption slowed down and sometimes even decreased. It is a general awareness and respect for the environment that has helped to reduce this almost exponential growth. Switzerland is indeed in an energy transition that is just beginning.

Secondly, we will see by year the evolution of the enery production in Switzerland in %

In [ ]:
shared_plot(final_data_perc)

With these graphs, the result is a lot more striking. Indeed, we can see that coal accounted for 80% of energy production in 1900 and today nothing at all and it's the same for the wood and charcoal. We can also see that oil accounts for 80% of energy consumption in 1970. The reason is that oil at that time was cheap energy and there was plenty of it. This is why oil will gradually dethrone coal in countries where it is rare. Switzerland was not apart and was following the technological advances and unfortunately did not pass through the oil crisis in the 1970s.  That's why it will reduce its oil consumption and find other energies. <br> 
That's at this moment that we can see an increase in gas and nuclear energy. Finally we can see that today Switzerland is in an energetic transition because the renewable energies start to be used and the consumption of nuclear and oil continues to decrease <br> 
What is also striking is that already in 1940, the hydrolic force was already used and accounted for 40% of energy consumption in Switzerland. However, if we only analyze this graph we can say that the production of hydrolic energy has decreased. But this is not the case because thanks to the graphs above, we can see that the production of hydrolic energy in TJ continues to increase. Unfortunately with such energy consumption, hydropower alone is not enough and that is why in our graphs, this consumption decreases. It is therefore important to study the graphs at the same time so as not to get false results

In [ ]:
final_data_perc.index.name = "Year"
final_data_tj.index.name = "Year"
final_data_tj_full.index.name = "Year"

final_data_perc.to_csv('Data/final_data_perc.csv')
final_data_tj.to_csv('Data/final_data_tj.csv')
final_data_tj_full.to_csv('Data/final_data_tj_full.csv')

# The Evolution of the population

if we can find relation between the increase of the population and the increase of the energy consumption (see the ratio and what a lambda person "use" each year)

### Import data

In [ ]:
swiss_population_path = 'Data/population_Suisse.xlsx'
metadata_swiss_pop = pd.read_excel(swiss_population_path, index_col = 0, header = 3)

In [ ]:
metadata_swiss_pop.head()

### Clean Data

In [ ]:
metadata_swiss_pop2 = metadata_swiss_pop[3:]

In [ ]:
metadata_swiss_pop2.head()

In [ ]:
metadata_swiss_pop2.columns.values

We just need to keep the most important data and drop the others

In [ ]:
swiss_pop_drop = metadata_swiss_pop2.drop(['Naissances'], axis=1).drop(['Décès 3)'], axis=1).drop(['Accroissement'], axis=1).drop(['Immigrations 4)'], axis=1).drop(['Emigrations 4)'], axis=1).drop(['Solde'], axis=1).drop(['Acquisition de'], axis=1).drop(['Divergences'], axis=1)
swiss_pop_drop.head()

In [ ]:
#Change the name of colums to add unit, either % or TJ (Tera Joule)
def change_col_name(df, new_name):
    new_df = df.copy()
    cols_df = new_df.columns.values 
    for i in range(len(cols_df)):
        if(len(cols_df) == len(new_name)):
            name = cols_df[i]
            new_df = new_df.rename(columns={name: new_name[i]})
    return new_df

In [ ]:
new_name = ["Population January", "Population December", "Absolute values", "Values in %"]

swiss_pop_clean = change_col_name(swiss_pop_drop, new_name)
swiss_pop_clean.head()

We can see that at the end we have data that we don't what so like the previous one we will only keep the dates

In [ ]:
swiss_pop_cleaner = keep_only_dates(swiss_pop_clean)
swiss_pop_cleaner.head()

For the analysis we will take the values since 1910 (because our data start at the date)

In [ ]:
# So from 1861 to 1910 there are 49 dates so it's easy
final_swiss_pop = swiss_pop_cleaner.loc[swiss_pop_cleaner.index.values[49:]]
final_swiss_pop.head()

Now, we have good data and we can use them to analyse our previous data

## Analysis

Let's check our results now if we take the population of each December of each year

In [ ]:
pop_december = final_swiss_pop['Population December']
pop_december.head()

Attention the date is only until 2016 and not 2014 like the previsous data and the year 2010 is missing...

In [ ]:
pop_december.index.values

In [ ]:
def shared_plot_pop_energy(energy_df, pop_df):        
    fig, axs = plt.subplots(1, 2, figsize=(20,6))
    energy_df.plot(ax=axs[0], title = "Energy")
    pop_df.plot(ax=axs[1], title = "Population")
    
    plt.show()

In [ ]:
shared_plot_pop_energy(final_data_tj_full,pop_december)

We can see that there is a correlation which seems logical because the more the population increases, the more energy it consumes. However we can see that the curves are not the same. Indeed, the exponential growth of the energy graph from 1945 to 1975 is much more pronounced than the other one. From this we can conclude that the population plays a role but it is not the only factor, other factors mentioned above also come into play. Furthermore, we can see that the population continues to increase after the 2000s when energy consumption stabilizes (and even decreases). It is at this moment that one realizes that the population plays a role in the consumption of energy. Despite the fact that the population is growing, after the 2000s, a change in the moral level of global warming helps to stabilize energy consumption (a mentality that changed little between 1945 and 1990).

We can compare the percentage too

In [ ]:
# So we take the Values in % and plot them
pop_perc = final_swiss_pop['Values in %']
shared_plot_pop_energy(final_data_perc,pop_perc)

These graphs are speaking especially after 1960. We can see the demographic evolution of the population between 1940 and 1960. However after, the population does not grow anymore and decreases after 1975. We can understand this with the oil crisis which is decreasing at the same time in the graph of energy. We can also see from the 2000s, with the energetic transition, the people are more forward looking and regaining confidence so that the population also increases.

Now we can analyse the consumption of a lambda person over the years

In [ ]:
def person_consumption(df_people, df_consumption):
    new_df = []
    list_idx = df_consumption.index.values # Seen before, both have same index but the indexes in consumption 
                                            # goes to 2014 and the is not the year 2010 in the population data
    for idx in list_idx:
        if(idx != 2010):
            result = df_consumption.loc[idx][0] / df_people[idx]  
            new_df.append({'Date': idx, 'Consumption in TJ': result})
    return pd.DataFrame(new_df).set_index("Date")

In [ ]:
consumption_per_person = person_consumption(pop_december,final_data_tj_full)
consumption_per_person.head()

In [ ]:
consumption_per_person.plot()

Here it is very interesting because we can see the average consumption of a lambda person per year. We can see that we have the same growth as before but it tends to decrease since the 2000s. The reasons for this can be the same as before but after reflection, after the second world war, there was the birth of television for any public. This has pushed people to consume more and more energy because of, for example, appliances, phones or computers. This large-scale development is also to be taken into account because more and more people will consume energy. For example, we go from 0.025 Tera Joules in 1945 to 0.16 in 1990. People use more and more energy with an exponential increase. Fortunately today we can see that despite the fact that there are more and more people, consumption decreases and there is a real awareness and it is thanks to that also that the energetic transition can be accomplished.


Here is the text where we took some informations:[Energie Consumption](https://www.avenir-suisse.ch/files/2017/03/energie_pour_l_economie_et_le_bien_etre.pdf) <br>

We can see in this report some useful informations about the energy consumption in Switzerland. For example the daily consumption of a farmer passed 7000 years ago from 14 KWh to 118 KWh. (This represent 0.0004248 Tera Joules = 0.4248 Giga Joules). Compared to what can be analyzed above it means that there are people who consume more energy than others. <br> 

Furthermore, in the text, with a consumption of 7600 KWh (27.36 GJ  = 0.02736 TJ) per inhabitant, Switzerland was in the European average in 2008. That's more than what we have because we took an average of the total population and maybe in the text they chose specific people. Nevertheless, we have quite similar results with the text.

# The Nuclear production

In [ ]:
PRODUCTION_NAME = 'Production en Gwh'
ELEC_USED_NAME = 'Production utilisee (%)'

### Import Data

We start by importing the data

In [ ]:
nuclear_path = 'Data/nuclear_production.csv'
nuclear_data = pd.read_csv(nuclear_path, delimiter=",")

In [ ]:
nuclear_data.head(4)

The data contains in fact two kinds of data: electric production and % electricity used. We want to split both kinds of data in separate dataframe and index them by 'Centrale' and then by year using a multiindex. This way it will be easy to plot and see the evolution of production for each centrale through years

In [ ]:
# Reorder columns
re_cols = ['Power plant', 'Year', 'production/ used electricity', 'num']
nuclear_recols = nuclear_data[re_cols]
nuclear_recols.head(2)

In [ ]:
# Split the data in two categories: production and electricity used
def split_df(df):
    prod = nuclear_recols[::2].drop('production/ used electricity', axis=1)
    elec_used = nuclear_recols[1::2].drop('production/ used electricity', axis=1)
    return prod, elec_used

In [ ]:
production_df, elec_used_df = split_df(nuclear_recols)

In [ ]:
production_df.head(2)

In [ ]:
elec_used_df.head(2)

In [ ]:
# Create hierarchical index from a dataframe. cols are the columns that will become index, asc takes 1 or 0 per column
# whether you want to index in ascending order or not, valname is the name you want to give to the data
def index_by(cols, df, asc, valname):
    new_df = df.copy()
    new_df.set_index(cols, inplace = True)
    new_df.sort_index(level = cols, ascending = asc, inplace = True)
    new_df.columns = [valname]
    return new_df

In [ ]:
prod_idxd = index_by(['Power plant', 'Year'], production_df, [1,1], PRODUCTION_NAME)
prod_idxd.head(3)

In [ ]:
elec_idxd = index_by(['Power plant', 'Year'], elec_used_df, [1,1], ELEC_USED_NAME)
elec_idxd.head(3)

In [ ]:
# Remove 'Total' column from df and return it
def remove_total(df):
    total = elec_idxd.loc['Total']
    df.drop('Total', inplace = True)
    return total

In [ ]:
total_prod = remove_total(prod_idxd)

In [ ]:
total_elec = remove_total(elec_idxd)

In [ ]:
# Plots the dataframe values in two plots (half the data in each)
def plot_by_year(valname, df):
    df_unstacked = df.unstack().T.loc[valname]
    middle_idx = int(len(df_unstacked.columns)/2)
    split_val1 = df_unstacked.columns[middle_idx]
    split_val2 = df_unstacked.columns[middle_idx + 1]
    x = df_unstacked.loc[:, :split_val1]
    y = df_unstacked.loc[:, split_val2:]
    f, (ax1, ax2) = plt.subplots(1, 2, figsize = (14,5), sharey = True)
    plt.figure(); 
    x.plot(ax=ax1)
    y.plot(ax = ax2)
    #ax1.set_title('Sharing Y axis')

In [ ]:
plot_by_year(PRODUCTION_NAME, prod_idxd)

In [ ]:
plot_by_year(ELEC_USED_NAME, elec_idxd)

We can explain the different peaks by looking in the [report](http://www.nuklearforum.ch/sites/default/files/media/Chronique_fr_2000_2005_0.pdf) of events related to Swiss power plants between 2000 and 2005 
For example, the biggest peak in the Leibstadt power plant is explained by the following quote:

Original quote in french
"(5043) 28 mars 2005 : une perturbation survenue à l’intérieur de l’alternateur entraîne un arrêt de la turbine de la
centrale nucléaire de Leibstadt. Le réacteur, qui n’est pas touché par l’événement, est ensuite mis à l’arrêt par l’équipe
d’exploitation."

Here is the list of the reports accessible from the [nuklearforum](http://www.nuklearforum.ch/fr/chronique_nucleaire) website  
[1975 - 1999](https://www.nuklearforum.ch/sites/default/files/media/Chronik_dt_fr_bis_1999.pdf)  
[2000 - 2005](http://www.nuklearforum.ch/sites/default/files/media/Chronique_fr_2000_2005_0.pdf)  
[2006 - 2011](http://www.nuklearforum.ch/sites/default/files/media/Chronik_fr_ab_2006_0.pdf)  
[2012 - 2013](https://www.nuklearforum.ch/sites/default/files/page/Chronik_fr_ab-01-01-2012%20bis%2031-12-2013.pdf)  
[2014](https://www.nuklearforum.ch/sites/default/files/page/Chronik_fr_des-01-01-2014.pdf)  


In [ ]:
prod_idxd.to_csv('Data/prod_nuc_Ghw.csv')
elec_idxd.to_csv('Data/prod_nuc_perc.csv')

trans_prod_idxd = prod_idxd.unstack().T.loc["Production en Gwh",:]
trans_prod_idxd.columns.name = ""
trans_elec_idxd = elec_idxd.unstack().T.loc["Production utilisee (%)",:]
trans_elec_idxd.columns.name = ""

trans_prod_idxd.to_csv('Data/trans_prod_nuc_Ghw.csv')
trans_elec_idxd.to_csv('Data/trans_prod_nuc_perc.csv')